In [7]:
#!pip install optuna

In [8]:
#!pip install xgboost

In [9]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [10]:
df=pd.read_csv("http://raw.githubusercontent.com/sunnysavita10/SVM-Kernel-Python-Implementation/refs/heads/main/Admission_Prediction.csv")

In [11]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         500 non-null    int64  
 1   GRE Score          485 non-null    float64
 2   TOEFL Score        490 non-null    float64
 3   University Rating  485 non-null    float64
 4   SOP                500 non-null    float64
 5   LOR                500 non-null    float64
 6   CGPA               500 non-null    float64
 7   Research           500 non-null    int64  
 8   Chance of Admit    500 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 35.3 KB


In [13]:
df.shape

(500, 9)

In [14]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Serial No.,500.0,250.500000,144.481833,1.00,125.7500,250.50,375.25,500.00
GRE Score,485.0,316.558763,11.274704,290.00,308.0000,317.00,325.00,340.00
TOEFL Score,490.0,107.187755,6.112899,92.00,103.0000,107.00,112.00,120.00
University Rating,485.0,3.121649,1.146160,1.00,2.0000,3.00,4.00,5.00
SOP,500.0,3.374000,0.991004,1.00,2.5000,3.50,4.00,5.00
LOR,500.0,3.484000,0.925450,1.00,3.0000,3.50,4.00,5.00
CGPA,500.0,8.576440,0.604813,6.80,8.1275,8.56,9.04,9.92
Research,500.0,0.560000,0.496884,0.00,0.0000,1.00,1.00,1.00
Chance of Admit,500.0,0.721740,0.141140,0.34,0.6300,0.72,0.82,0.97


In [16]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [17]:
df['GRE Score']=df['GRE Score'].fillna(df['GRE Score'].median())
df['TOEFL Score']=df['TOEFL Score'].fillna(df['TOEFL Score'].median())
df['University Rating']=df['University Rating'].fillna(df['University Rating'].median())

In [18]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [19]:
X=df.drop(['Serial No.','Chance of Admit'],axis=1)


In [20]:
y=df['Chance of Admit']

In [21]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=20)


In [22]:
std_sca=StandardScaler()
transformed_X_train=std_sca.fit_transform(X_train)
transformed_X_test=std_sca.transform(X_test)

In [ ]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def objective(trial, data=X, target=y):
    train_x, test_x, train_y, test_y = train_test_split(X, y, random_state=25, test_size=0.2)
    param = {
        # 'tree_method': 'gpu_hist',  # Remove or comment out this line to use CPU
        'tree_method': 'hist',  # Use 'hist' for CPU or 'gpu_hist' if you have a compatible GPU and drivers installed
        'lambda': trial.suggest_float('lambda', 1e-4, 10.0),
        'alpha': trial.suggest_float('alpha', 1e-4, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [.1, .2, .3, .4, .5, .6, .7, .8, .9, 1]),
        'subsample': trial.suggest_categorical('subsample', [.1, .2, .3, .4, .5, .6, .7, .8, .9, 1]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.00001, 0.0003, 0.0008, 0.5, 0.6, 1, 8]),
        'n_estimators': 3000,
        'max_depth': trial.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state': trial.suggest_categorical('random_state', [10, 20, 30, 2000, 3454, 243123]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 200),
    }
    Xgb_reg_model = xgb.XGBRegressor(**param)
    Xgb_reg_model.fit(train_x,train_y,eval_set=[(test_x,test_y)],verbose=True)
    y_pred = Xgb_reg_model.predict(test_x)
    mse = mean_squared_error(test_y, y_pred)
    return mse


ModuleNotFoundError: No module named 'optuna'

In [ ]:
find_param=optuna.create_study()
find_param.optimize(objective,n_trials=12)

In [ ]:
find_param.best_trial.params

In [ ]:
find_param.trials_dataframe()

In [ ]:
optuna.visualization.plot_optimization_history(find_param)

In [ ]:
best_params={
  'lambda': 1.0449881560787229,
 'alpha': 0.32412877649423805,
 'colsample_bytree': 0.9,
 'subsample': 0.8,
 'learning_rate': 1,
 'max_depth': 7,
 'random_state': 3454,
 'min_child_weight': 105}

In [ ]:
model=xgb.XGBRegressor(**best_params)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)


In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model2=RandomForestRegressor()
model2.fit(X_train,y_train)
y_pred2=model2.predict(X_test)
r2_score(y_test,y_pred2)